# Projeto 1 - Ciência dos Dados

Nome: Raphael Lahiry de Barros

Nome: Florencia Averame Kramer

# O que temos que fazer?
precisamos criar um dataframe para cada relevância   OK

dentro desses dataframes separar as frases em palavras OK   

analisar a freq relativa de cada palavra --> probabilidade daquela palavra pertencer àquela relevância    

quando quisermos avaliar o teste--> analisamos a probabilidade de cada palavra de uma frase pertencer a uma relevância (usar o smoothing -- aula 8 -- para as palavras que talvez não existam em certas categorias)  

com isso, para cada palavra teremos a probabilidade de que esta pertença a cada relevância  
a relevância com maior probabilidade será a daquela palavra

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
## Carregando algumas bibliotecas:

In [27]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [28]:
import os

filename = 'ps4.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo ps4.xlsx, tudo certo para prosseguir com a prova!


### Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [29]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,@odonodocabare_ comprei um tablet dia desses e...,0.0
1,@encanadorchapad o negócio é o ps4 slim. peque...,4.0
2,rt @ps3brasil: tohu é anunciado para ps4; deta...,2.0
3,dividindo essa semana entre dois jogos no modo...,0.0
4,@leobruto @fridaygus @douglasxavier8 @ocerveje...,0.0


In [30]:
train['Treinamento'] = train['Treinamento'].astype('str')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto é o playstation 4. Consideramos relevantes as mensagens que falavam sobre questões técnicas do produto e sobre satisfação, sejam positivas ou negativas. Consideramos irrelevantes as mensagens que só mencionavam o nosso produtos nas '#' ou que eram opiniões pessoais que não agregavam na nossa análise.

___
## Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [33]:
#função que apaga os caracteres especiais e as menções de usuários
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;[]]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    #filtro que remove as menções à usuários com @
    filtro = filter(lambda x:x[0]!='@', text_subbed.split())
    juncao = " ".join(filter(lambda x:x[0]!='@', text_subbed.split()))
    
    return juncao
    

In [34]:
#aplicando a função cleanup na base treinamento e deixando tudo em letra minúscula
train['Treinamento'] = train['Treinamento'].apply(cleanup)
train['Treinamento'] = train['Treinamento'].str.lower()

In [35]:
train['Relevância'] = train['Relevância'].astype('category')

In [36]:
train['Relevância'].cat.categories = ['Muito irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito relevante']
train['Relevância'].cat.as_ordered()

0      Muito irrelevante
1        Muito relevante
2                 Neutro
3      Muito irrelevante
4      Muito irrelevante
             ...        
746          Irrelevante
747          Irrelevante
748               Neutro
749    Muito irrelevante
750          Irrelevante
Name: Relevância, Length: 751, dtype: category
Categories (5, object): [Muito irrelevante < Irrelevante < Neutro < Relevante < Muito relevante]

### Criando os filtros para criar os dataFrames por relevância

In [37]:
#ligando a relevância à um número
muito_relevante = train['Relevância'] == 'Muito relevante'
relevante = train['Relevância'] == 'Relevante'
neutro = train['Relevância'] == 'Neutro'
irrelevante = train['Relevância'] == 'Irrelevante'
muito_irrelevante = train['Relevância'] == 'Muito irrelevante'

### Criando os dataFrames separados por relevância

In [38]:
df_muito_relevante = train.loc[muito_relevante, :]
df_relevante = train.loc[relevante, :]
df_neutro = train.loc[neutro, :]
df_irrelevante = train.loc[irrelevante, :]
df_muito_irrelevante = train.loc[muito_irrelevante, :]

### Separando cada dataFrame da sua respectiva relevância em listas de palavras

In [39]:
palavras_muito_relevantes = []
lista_vrw = []
for vrw in df_muito_relevante['Treinamento']:
    palavras_muito_relevantes.append(vrw.split(' '))
for vrl in palavras_muito_relevantes:
    lista_vrw += vrl

In [40]:
palavras_relevantes = []
lista_rw = []
for rw in df_relevante['Treinamento']:
    palavras_relevantes.append(rw.split(' '))
for rl in palavras_relevantes:
    lista_rw += rl

In [41]:
palavras_neutras = []
lista_nw = []
for nw in df_neutro['Treinamento']:
    palavras_neutras.append(nw.split(' '))
for nl in palavras_neutras:
    lista_nw += nl

In [42]:
palavras_irrelevantes = []
lista_iw = []
for iw in df_irrelevante['Treinamento']:
    palavras_irrelevantes.append(iw.split(' '))
for il in palavras_irrelevantes:
    lista_iw += il

In [43]:
palavras_muito_irrelevantes = []
lista_viw = []
for viw in df_muito_irrelevante['Treinamento']:
    palavras_muito_irrelevantes.append(viw.split(' '))
for vil in palavras_muito_irrelevantes:
    lista_viw += vil

### Transformando as listas de palavras de cada relevância em pd.Series

In [44]:
serie_muito_relevante = pd.Series(lista_vrw)
serie_relevante = pd.Series(lista_rw)
serie_neutro = pd.Series(lista_nw)
serie_irrelevante = pd.Series(lista_iw)
serie_muito_irrelevante = pd.Series(lista_viw)

### Frequências relativas e absolutas das palavras de cada relevância

In [76]:
#muito relevante
freq_ab_muito_relevante = serie_muito_relevante.value_counts() 
freq_re_muito_relevante = serie_muito_relevante.value_counts(True) + (alpha/lingua_portuguesa)

In [77]:
#relevante
freq_ab_relevante = serie_relevante.value_counts()
freq_re_relevante = serie_relevante.value_counts(True) + (alpha/lingua_portuguesa)

In [78]:
#neutro
freq_ab_neutro = serie_neutro.value_counts()
freq_re_neutro = serie_neutro.value_counts(True) + (alpha/lingua_portuguesa)

In [79]:
#irrelevante
freq_ab_irrelevante = serie_irrelevante.value_counts()
freq_re_irrelevante = serie_irrelevante.value_counts(True) + (alpha/lingua_portuguesa)

In [80]:
#muito irrelevante
freq_ab_muito_irrelevante = serie_muito_irrelevante.value_counts()
freq_re_muito_irrelevante = serie_muito_irrelevante.value_counts(True) + (alpha/lingua_portuguesa)

### Definindo a quantidade total de palavras na língua portuguesa

In [75]:
#total_train são as palavras existentes nos tweets de treinamento, nos quais verificaremos
#a freq relativa e somaremos um alpha (que representa que a palavra apareceu uma vez) pra garantir
#que as palavras que não existem nos tweets classificados não zerem o classificador
total_train = lista_vrw + lista_rw + lista_nw + lista_iw + lista_viw

#na fórmula precisamos dividir pela quantidade de palavras existentes na lingua portuguesa
#para obter esse valor somamos a quantidade de palavras existentes nos tweets classificados 
#com um "chute" das palavras existentes que não estão em nenhum tweet classificado
alpha = 1
v = 1e6
lingua_portuguesa = len(total_train) + alpha*v

### Pd.series das palavras que aparecem nos tweets classificados

In [62]:
serie_total_train = pd.Series(total_train)
#serie_total_train.value_counts()

## Montando Naive-Bayes

### Classificador da relevância de cada palavra

In [84]:
# lista muito relevante

def classificador_relev_palavra(palavra):
    
    dict_relevancia_palavras = {}
    
    if palavra not in total_train:
        dict_relevancia_palavras[palavra] = (alpha/lingua_portuguesa)
        print('NÂO EXISTIA DENTRO DE TRAIN')
        
    else:
        print("ENTROU NO ELSE")
        mr=0
        r=0
        n=0
        i=0
        mi=0
            
        if palavra in lista_vrw:
            mr = freq_re_muito_relevante[palavra] 

        if palavra in lista_rw:
            r = freq_re_relevante[palavra] 

        if palavra in lista_nw:
            n = freq_re_neutro[palavra]

        if palavra in lista_iw:
            i = freq_re_irrelevante[palavra] 

        if palavra in lista_viw:
            mi = freq_re_muito_irrelevante[palavra] 

        if (mr > r) and (mr > n) and (mr > i) and (mr > mi):
            print('mr')
            dict_relevancia_palavras[palavra] = "Muito relevante"

        elif (r > mr) and (r > n) and (r > i) and (r > mi):
            print('r')
            dict_relevancia_palavras[palavra] = "Relevante"

        elif (n > mr) and (n > r) and (n > i) and (n > mi):
            print('n')
            dict_relevancia_palavras[palavra] = "Neutro"

        elif (i > mr) and (i > r) and (i > n) and (i > mi):
            print('i')
            dict_relevancia_palavras[palavra] = "Irrelevante"

        elif (mi > mr) and (mi > r) and (mi > n) and (mi > i):
            print('mi')
            dict_relevancia_palavras[palavra] = "Muito irrelevante"

    return dict_relevancia_palavras[palavra]

classificador_relev_palavra('amigos')

ENTROU NO ELSE
r


'Relevante'

### Criando a probabilidade de cada relevância à priori

In [83]:
p_mr = len(serie_muito_relevante)/len(serie_total_train)
p_r = len(serie_relevante)/len(serie_total_train)
p_n = len(serie_neutro)/len(serie_total_train)
p_i = len(serie_irrelevante)/len(serie_total_train)
p_mi = len(serie_muito_irrelevante)/len(serie_total_train)

#print(p_mr + p_r + p_n + p_i + p_mi)

### Criando a função que retorna a probabilidade de uma frase pertencer à uma cateogria específica de relevância com base nas palavras que à compõem

#### 1º Prob (frase | relevância)
#### 2º Prob (relevância | frase)

In [ ]:
def datawitter(frase):
    
    palavras = frase.split(' ')
    
    mr = (alpha/lingua_portuguesa)
    r = (alpha/lingua_portuguesa)
    n = (alpha/lingua_portuguesa)
    i = (alpha/lingua_portuguesa)
    mi = (alpha/lingua_portuguesa)

    
    for palavra in palavras:
     
        if palavra not in total_train:
            mr *= (alpha/lingua_portuguesa)
            r *= (alpha/lingua_portuguesa)
            n *= (alpha/lingua_portuguesa)
            i *= (alpha/lingua_portuguesa)
            mi *= (alpha/lingua_portuguesa)
            print('NÂO EXISTIA DENTRO DE TRAIN')

        else:
            print("ENTROU NO ELSE")

            if palavra in lista_vrw:
                mr *= freq_re_muito_relevante[palavra] 

            if palavra in lista_rw:
                r *= freq_re_relevante[palavra] 

            if palavra in lista_nw:
                n *= freq_re_neutro[palavra]

            if palavra in lista_iw:
                i *= freq_re_irrelevante[palavra] 

            if palavra in lista_viw:
                mi *= freq_re_muito_irrelevante[palavra] 

                
    if (mr > r) and (mr > n) and (mr > i) and (mr > mi):
        print('mr')
        dict_relevancia_palavras[palavra] = "Muito relevante"

    elif (r > mr) and (r > n) and (r > i) and (r > mi):
        print('r')
        dict_relevancia_palavras[palavra] = "Relevante"

    elif (n > mr) and (n > r) and (n > i) and (n > mi):
        print('n')
        dict_relevancia_palavras[palavra] = "Neutro"

    elif (i > mr) and (i > r) and (i > n) and (i > mi):
        print('i')
        dict_relevancia_palavras[palavra] = "Irrelevante"

    elif (mi > mr) and (mi > r) and (mi > n) and (mi > i):
        print('mi')
        dict_relevancia_palavras[palavra] = "Muito irrelevante"

    return dict_relevancia_palavras[palavra]

    classificador_relev_palavra('amigos')

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [67]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test['Teste'] = test['Teste'].astype('str')
test['Teste'] = test['Teste'].apply(cleanup)
test['Teste'] = test['Teste'].str.lower()

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**